In [12]:
import pandas as pd
import psycopg2

#### postgresql connection credentials

#connection = psycopg2.connect(
#    host="localhost",
#    user="postgres",
#    password="vijay45",
#    database="postgres"

#)

#cursor=connection.cursor()

{ *** 3. COLLECT THE DOUBLES COMPETITOR RANKINGS DATA FROM THE API ENDPOINTS ***  }


In [13]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=3cx83hgyhyf1YOaeIdPIxYu10bw5H0JSANgpnVSI"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)



FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR DATA


In [14]:

if response.status_code == 200:
    try:
        data = response.json()

        ab = []
        auto_increment_id = 1

        # Iterate over rankings to extract the required data
        for ranking in data.get("rankings", []):
            for competitor_ranking in ranking.get("competitor_rankings", []):
                competitor = competitor_ranking.get("competitor", {})
                ab.append({
                    "rank_id": auto_increment_id,
                    "rank": competitor_ranking.get("rank"),
                    "movement": competitor_ranking.get("movement"),
                    "points": competitor_ranking.get("points"),
                    "competitions_played": competitor_ranking.get("competitions_played"),
                    "competitor_id": competitor.get("id"),
                    "name": competitor.get("name"),
                    "country": competitor.get("country"),
                    "week": ranking.get("week"),  # Adjusted to get 'week' from the parent ranking object
                    "country_code": competitor.get("country_code"),
                    "abbreviation": competitor.get("abbreviation"),
                })
                auto_increment_id += 1
    except Exception as e:
        print(f"Error processing data: {e}")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")


data =pd.DataFrame(ab)
data


,rank_id,rank,movement,points,competitions_played,competitor_id,name,country,week,country_code,abbreviation
0,1,1,0,7530,22,sr:competitor:49363,"Pavic, Mate",Croatia,6,HRV,PAV
1,2,1,0,7530,22,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,6,SLV,ARE
2,3,3,0,7075,30,sr:competitor:637970,"Patten, Henry",Great Britain,6,GBR,PAT
3,4,4,0,7055,28,sr:competitor:14898,"Heliovaara, Harri",Finland,6,FIN,HEL
4,5,5,0,6475,21,sr:competitor:87690,"Thompson, Jordan",Australia,6,AUS,THO
...,...,...,...,...,...,...,...,...,...,...,...
995,996,496,1,124,4,sr:competitor:32862,"Diatchenko, Vitalia",Neutral,6,None,DIA
996,997,497,-38,124,14,sr:competitor:230108,"McGiffin, Tenika",Australia,6,AUS,MCG
997,998,498,0,124,15,sr:competitor:720627,"Senic, Natalija",Serbia,6,SRB,SEN
998,999,499,0,124,21,sr:competitor:360054,"Allen, Kolie",USA,6,USA,ALL


In [16]:
# Establish the database connection

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="vijay45",
    database="postgres"
)
cursor = connection.cursor()

# CREATING SQL TABLE FOR [COMPETITOR_RANKING_TABLE]
cursor.execute("""
    CREATE TABLE IF NOT EXISTS competitor_ranking_table (
            rank_id INT ,
            rank INT,
            movement INT,
            points INT,
            competitions_played INT,
            competitor_id VARCHAR(255)
    )
""")
# CREATING SQL TABLE FOR [COMPETITORS_TABLE]

cursor.execute("""
    CREATE TABLE IF NOT EXISTS competitors_table (
        competitor_id VARCHAR(255),
        name VARCHAR(255),
        country VARCHAR(255),
        week VARCHAR(50),       
        country_code VARCHAR(10),
        abbreviation VARCHAR(10)
    )
""")

# inserting the data into postgreSQL

for _, row in data.iterrows():
    cursor.execute(
        """
        INSERT INTO competitor_ranking_table (rank_id, rank , movement, points, competitions_played, competitor_id)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (row["rank_id"], row["rank"], row["movement"], row["points"], row["competitions_played"], row["competitor_id"])
    )


competitors_df = data.drop_duplicates(subset="competitor_id")[["competitor_id", "name", "country","week", "country_code", "abbreviation"]]

# inserting the data into postgreSQL

for _, row in data.iterrows():
    cursor.execute(
        """
        INSERT INTO competitors_table (competitor_id, name, country, week, country_code, abbreviation)
        VALUES (%s, %s, %s, %s,%s, %s)
        """,
        (row["competitor_id"], row["name"], row["country"],row["week"],row["country_code"], row["abbreviation"])
    )

connection.commit()
connection.close()

print("Data has been inserted into POSTGRESQL!")

Data has been inserted into POSTGRESQL!
